# Frequency

In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

feature_pos = [[0,8],[8,12],[12,28],[28,44],[44,52],[52,64],[64,80],[80,92],[92,102]] 
df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

## Save Frequency Features
Skip this if you have saved the features

In [ ]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [4]:
feat = dataframes[0].shape[1]
# feat = 12

In [5]:
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        mat = np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0)
        mat = np.squeeze(np.asarray(mat))
        encodedData.append(mat)
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

def one_hot_encoding_feature(encoders, dfs, feature):
    if len(encoders) == 0:
        encoder, data = one_hot_encoding(None, dfs, feature_pos[feature][0])
        encoders.append(encoder)    
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            encoder, dataCol = one_hot_encoding(None, dfs, col)
            encoders.append(encoder)
            data = np.concatenate((data, dataCol), axis=1)
    else:
        encoder, data = one_hot_encoding(encoders[0], testdatas, feature_pos[feature][0])
        for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
            i = col - feature_pos[feature][0]
            encoder, dataCol = one_hot_encoding(encoders[i], testdatas, col)
            data = np.concatenate((data, dataCol), axis=1)
    return encoders, data

In [ ]:
import gc

# Give up last feature
for feat in range(len(feature_pos) - 1):
    print(feat)
    encoders, train = one_hot_encoding_feature([], dataframes, feat)
    encoders, test = one_hot_encoding_feature(encoders, testdatas, feat)
    np.save('freq/' + str(feat) + '_train.npy', train)
    np.save('freq/' + str(feat) + '_test.npy', test)
    gc.collect()

In [4]:
from tqdm import tqdm

def handle_integers(dfs, col):
    data = []
    for df in dfs:
        sumCol = np.sum(df[:,col], axis=0).reshape(1, -1)
        meanCol = np.mean(df[:, col], axis=0).reshape(1, -1)
        dataCol = np.concatenate((sumCol, meanCol), axis=1)
        # dataCol = sumCol
        # print(dataCol.shape)
        data.append(dataCol)
    data = np.concatenate(data, axis=0)    
    # print(data.shape)
    return data

def handle_integers_feature(dfs, feature):
    if feature != 8:
        return None
    data = handle_integers(dfs, feature_pos[feature][0])
    for col in tqdm(range(feature_pos[feature][0] + 1,feature_pos[feature][1])):
        dataCol = handle_integers(dfs, col)
        data = np.concatenate((data, dataCol), axis=1)
    return data

train = handle_integers_feature(dataframes, 8)
test = handle_integers_feature(testdatas, 8)
np.save('freq/8_train.npy', train)
np.save('freq/8_test.npy', test)

100%|█████████████████████████████████████████████████████████████████████████| 9/9 [00:02<00:00,  4.41it/s]


## Load Frequency Features

In [3]:
import numpy as np
feature_selected = [True,True,True,True,True,True,True,True,False] 

def load_freq(feat, postfix):
    data = np.load("freq/{}_{}.npy".format(feat, postfix))
    return data

def load_test_freq(feat):
    return load_freq(feat, 'test')

def load_train_freq(feat):
    return load_freq(feat, 'train')

In [4]:
def select_features(feature_selected):
    XTrain = []
    XTest = []
    for feat in range(len(feature_pos)):
        if feature_selected[feat] == False:
            continue
        train = load_train_freq(feat)
        test = load_test_freq(feat)
        XTrain.append(train)
        XTest.append(test)
    XTrain = np.concatenate(XTrain, axis=1)
    XTest = np.concatenate(XTest, axis=1)
    return XTrain, XTest

def all_not_selected(feature_selected):
    for i in range(len(feature_pos)):
        if feature_selected[i] == True:
            return False
    return True

In [12]:
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import metrics
import gc

scores = [0]
f0s = [0]
f1s = [0]
f2s = [0]
f3s = [0]
f4s = [0]
f5s = [0]
f6s = [0]
f7s = [0]
score = (0, 0, 0, 0, 0, 0, 0, 0, 0)
cnt = 0
for f0 in range(2):
    feature_selected[0] = bool(f0)
    for f1 in range(2):
        feature_selected[1] = bool(f1)
        for f2 in range(2):
            feature_selected[2] = bool(f2)
            for f3 in range(2):
                feature_selected[3] = bool(f3)
                for f4 in range(2):
                    feature_selected[4] = bool(f4)
                    for f5 in range(2):
                        feature_selected[5] = bool(f5)
                        for f6 in range(2):
                            feature_selected[6] = bool(f6)
                            for f7 in range(2):
                                cnt += 1
                                feature_selected[7] = bool(f7)
                                if all_not_selected(feature_selected) == True:
                                    continue
                                if cnt % 10 == 0:
                                    print(cnt)
                                XTrain, XTest = select_features(feature_selected)
                                del XTest
                                XTrain, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
                                gc.collect()
                                model = LGBMClassifier()
                                model.fit(XTrain, y_train)
                                pred = model.predict_proba(X_valid)[:,1]
                                fpr, tpr, thresholds = metrics.roc_curve(y_valid, pred)
                                auc_score = metrics.auc(fpr, tpr)
                                scores.append(auc_score)
                                f0s.append(f0)
                                f1s.append(f1)
                                f2s.append(f2)
                                f3s.append(f3)
                                f4s.append(f4)
                                f5s.append(f5)
                                f6s.append(f6)
                                f7s.append(f7)
                                gc.collect()
                                if score[0] < auc_score:
                                    score = (auc_score, f0, f1, f2, f3, f4, f5, f6, f7)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250


In [17]:
auc_csv = pd.DataFrame({'f0': f0s, 'f1': f1s, 'f2': f2s, 'f3': f3s, 'f4': f4s, 'f5': f5s, 'f6': f6s, 'f7': f7s, 'auc': scores}).sort_values(by=['auc'])
auc_csv.to_csv('freq_auc.csv')
auc_csv

,f0,f1,f2,f3,f4,f5,f6,f7,auc
0,0,0,0,0,0,0,0,0,0.000000
2,0,0,0,0,0,0,1,0,0.882728
1,0,0,0,0,0,0,0,1,0.889711
3,0,0,0,0,0,0,1,1,0.895471
5,0,0,0,0,0,1,0,1,0.980764
4,0,0,0,0,0,1,0,0,0.980828
6,0,0,0,0,0,1,1,0,0.981217
7,0,0,0,0,0,1,1,1,0.982000
11,0,0,0,0,1,0,1,1,0.982529
18,0,0,0,1,0,0,1,0,0.982658


In [15]:
print(score)

(0.992238129804707, 1, 1, 1, 1, 0, 0, 0, 0)


In [16]:
for i in range(8):
    feature_selected[i] = bool(score[i + 1])

XTrain, XTest = select_features(feature_selected)

## Try Others

In [5]:
feature_selected = [True,True,True,False,True,False,False,False,True] 
XTrain, XTest = select_features(feature_selected)

## Train models

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 69229)
(3733, 69229)
(14929,)
(3733,)


In [7]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train, y_train)
valid_set = lgb.Dataset(X_valid, y_valid)

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [10]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = np.log(1 - best_score)
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials,
            rstate=np.random.RandomState(1024) # fixing random state for the reproducibility
           )
best

[1]	valid_0's auc: 0.951626                                                                                 
Training until validation scores don't improve for 5 rounds                                                 
[2]	valid_0's auc: 0.965879                                                                                 
[3]	valid_0's auc: 0.966767                                                                                 
[4]	valid_0's auc: 0.968925                                                                                 
[5]	valid_0's auc: 0.969412                                                                                 
[6]	valid_0's auc: 0.970136                                                                                 
[7]	valid_0's auc: 0.971087                                                                                 
[8]	valid_0's auc: 0.935726                                                                                 
[9]	valid_0's auc: 

C:\Users\jw.chan\AppData\Local\Continuum\anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')



[2]	valid_0's auc: 0.970116                                                                                 
[3]	valid_0's auc: 0.972916                                                                                 
[4]	valid_0's auc: 0.97496                                                                                  
[5]	valid_0's auc: 0.976984                                                                                 
[6]	valid_0's auc: 0.978204                                                                                 
[7]	valid_0's auc: 0.97911                                                                                  
[8]	valid_0's auc: 0.979218                                                                                 
[9]	valid_0's auc: 0.980304                                                                                 
[10]	valid_0's auc: 0.980338                                                                                
[11]	valid_0's auc:

[27]	valid_0's auc: 0.979363                                                                                
[28]	valid_0's auc: 0.979361                                                                                
[29]	valid_0's auc: 0.979643                                                                                
[30]	valid_0's auc: 0.979989                                                                                
[31]	valid_0's auc: 0.980061                                                                                
[32]	valid_0's auc: 0.980508                                                                                
[33]	valid_0's auc: 0.980896                                                                                
[34]	valid_0's auc: 0.981352                                                                                
[35]	valid_0's auc: 0.981297                                                                                
[36]	valid_0's auc:

Did not meet early stopping. Best iteration is:                                                             
[50]	valid_0's auc: 0.988617
[1]	valid_0's auc: 0.962195                                                                                 
[2]	valid_0's auc: 0.970542                                                                                 
[3]	valid_0's auc: 0.974289                                                                                 
[4]	valid_0's auc: 0.975094                                                                                 
[5]	valid_0's auc: 0.975091                                                                                 
[6]	valid_0's auc: 0.975923                                                                                 
[7]	valid_0's auc: 0.97718                                                                                  
[8]	valid_0's auc: 0.977101                                                                        

...

[10]	valid_0's auc: 0.982597                                                                                
[11]	valid_0's auc: 0.983346                                                                                
[12]	valid_0's auc: 0.983753                                                                                
[13]	valid_0's auc: 0.983932                                                                                
[14]	valid_0's auc: 0.984699                                                                                
[15]	valid_0's auc: 0.985133                                                                                
[16]	valid_0's auc: 0.985268                                                                                
[17]	valid_0's auc: 0.985739                                                                                
[18]	valid_0's auc: 0.986158                                                                                
[19]	valid_0's auc:

[33]	valid_0's auc: 0.989021                                                                                
[34]	valid_0's auc: 0.989427                                                                                
[35]	valid_0's auc: 0.989263                                                                                
[36]	valid_0's auc: 0.9893                                                                                  
[37]	valid_0's auc: 0.989536                                                                                
[38]	valid_0's auc: 0.989526                                                                                
[39]	valid_0's auc: 0.989875                                                                                
[40]	valid_0's auc: 0.989941                                                                                
[41]	valid_0's auc: 0.98994                                                                                 
[42]	valid_0's auc:

[4]	valid_0's auc: 0.970763                                                                                 
[5]	valid_0's auc: 0.97303                                                                                  
[6]	valid_0's auc: 0.975187                                                                                 
[7]	valid_0's auc: 0.976255                                                                                 
[8]	valid_0's auc: 0.977073                                                                                 
[9]	valid_0's auc: 0.978131                                                                                 
[10]	valid_0's auc: 0.979573                                                                                
[11]	valid_0's auc: 0.980206                                                                                
[12]	valid_0's auc: 0.980703                                                                                
[13]	valid_0's auc:

[26]	valid_0's auc: 0.988816                                                                                
[27]	valid_0's auc: 0.989188                                                                                
[28]	valid_0's auc: 0.989328                                                                                
[29]	valid_0's auc: 0.989472                                                                                
[30]	valid_0's auc: 0.989529                                                                                
[31]	valid_0's auc: 0.989592                                                                                
[32]	valid_0's auc: 0.989696                                                                                
[33]	valid_0's auc: 0.989824                                                                                
[34]	valid_0's auc: 0.989812                                                                                
[35]	valid_0's auc:

{'boosting_type': 0,
 'colsample_by_tree': 0.6580377769618578,
 'learning_rate': 0.11765715157672925,
 'min_child_samples': 20.0,
 'num_leaves': 64.0,
 'reg_alpha': 0.4970782841472082,
 'reg_lambda': 0.0037197686817726505,
 'subsample_for_bin': 140000.0}

In [31]:
## Replace the dictionary with the best value and run this if the bottom code have error
best = {'boosting_type': 0,
 'colsample_by_tree': 0.6580377769618578,
 'learning_rate': 0.11765715157672925,
 'min_child_samples': 20.0,
 'num_leaves': 64.0,
 'reg_alpha': 0.4970782841472082,
 'reg_lambda': 0.0037197686817726505,
 'subsample_for_bin': 140000.0}
best

{'boosting_type': 0,
 'colsample_by_tree': 0.6580377769618578,
 'learning_rate': 0.11765715157672925,
 'min_child_samples': 20.0,
 'num_leaves': 64.0,
 'reg_alpha': 0.4970782841472082,
 'reg_lambda': 0.0037197686817726505,
 'subsample_for_bin': 140000.0}

In [32]:
from hyperopt import space_eval
best = space_eval(space, best)
best

{'boosting_type': 'gbdt',
 'colsample_bytree': 0.6580377769618578,
 'learning_rate': 0.11765715157672925,
 'metric': 'auc',
 'min_child_samples': 20.0,
 'num_leaves': 64.0,
 'reg_alpha': 0.4970782841472082,
 'reg_lambda': 0.0037197686817726505,
 'subsample_for_bin': 140000.0}

In [33]:
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
best['metric'] = 'auc'
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=lgbModel.best_iteration)

# Extract the best score
best_score = lgbModel.best_score['valid_0']['auc']

# Loss must be minimized
loss = np.log(1 - best_score)
print(loss)
YTest = lgbModel.predict(XTest)

[1]	valid_0's auc: 0.963594
Training until validation scores don't improve for 5 rounds
[2]	valid_0's auc: 0.978742
[3]	valid_0's auc: 0.98241
[4]	valid_0's auc: 0.983794
[5]	valid_0's auc: 0.984561
[6]	valid_0's auc: 0.985448
[7]	valid_0's auc: 0.986282
[8]	valid_0's auc: 0.986597
[9]	valid_0's auc: 0.987073
[10]	valid_0's auc: 0.987215
[11]	valid_0's auc: 0.987395
[12]	valid_0's auc: 0.987992
[13]	valid_0's auc: 0.988422
[14]	valid_0's auc: 0.988556
[15]	valid_0's auc: 0.988597
[16]	valid_0's auc: 0.988829
[17]	valid_0's auc: 0.988973
[18]	valid_0's auc: 0.989256
[19]	valid_0's auc: 0.989791
[20]	valid_0's auc: 0.989917
[21]	valid_0's auc: 0.990031
[22]	valid_0's auc: 0.990181
[23]	valid_0's auc: 0.990283
[24]	valid_0's auc: 0.990145
[25]	valid_0's auc: 0.990211
[26]	valid_0's auc: 0.990366
[27]	valid_0's auc: 0.990638
[28]	valid_0's auc: 0.990727
[29]	valid_0's auc: 0.990825
[30]	valid_0's auc: 0.99107
[31]	valid_0's auc: 0.991295
[32]	valid_0's auc: 0.991358
[33]	valid_0's auc: 0.9

In [34]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_bytree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [35]:
YTest = probs[:, 0]

In [36]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.001997
1,1,0.016329
2,2,0.997205
3,3,0.012417
4,4,0.497058
5,5,0.945905
6,6,0.997339
7,7,0.909380
8,8,0.001997
9,9,0.002005


In [37]:
df_test.to_csv('test.csv', index=False)